# Test 02: Principle Component Analysis

**Due: Friday 12/13 (by midnight)**

Name: Shruthi Madishetty

CWID: 50239178


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import sklearn
from scipy.optimize import minimize

In [ ]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10, 8) # set default figure size, 10in by 8in

## Question 1 

Load the MNIST data, and split it into a training
set, a validation set, and a test set (e.g., use the first 50,000 instances for
training, the next 10,000 for validation, and the last 10,000 for testing).
You can load the MNIST data from Scikit-Learn as shown in the cell given to you below.

Then train various classifiers, such as a Random Forest classifier, an Extra-
Trees classifier, and an SVM. Next, try to combine them into an ensemble
that outperforms them all on the validation set, using a soft or hard voting
classifier. Once you have found one, try it on the test set. How much better
does it perform compared to the individual classifiers?

## Step 1 - Load mnist data
    As part of this step,
    1. Loaded data into mnist variable
    2. Divided features into x and y as input and output

In [ ]:
from sklearn.datasets  import fetch_openml

mnist = fetch_openml('mnist_784')

In [ ]:
# Dividing x and y as input and output features for convenience, 'y' as mnist target and 'x' as remaining features
x = mnist.data
y = mnist.target.astype(np.int64)
print(x.shape)

## Step 2 - Split data

    As part of this step,There are total 70000 * 784 data in mnist
    1. Divided first 10000 rows into validation set ( as X_validation and y_validation)
    2. Then divided 10000 to Test set ( as X_test and y_test)
    3. Remaining 50000 goes to training dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train1, X_validation, y_train1, y_validation = train_test_split(x, y, test_size=10000, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size=10000, random_state=42)

## Step 3 - PCA (as fitering) to reduce features
    I am trying to find No. of components required for model since 784 features with 50000 samples are large dataset. And we need to reduce it to train the model with data.
    1. Finding No. of components with 50% variance taken using PCA.
    2. Transform Training set to the PCA components.


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.5).fit(X_train)
pca.n_components_

In [ ]:
train_components = pca.transform(X_train)
validate_components = pca.transform(X_validation)
test_components = pca.transform(X_test)

print(train_components.shape, y_train.shape, X_train.shape)

## Step 4 - Create models and fit 
    Creating 3 models(Random-forests, extra-tree-forest, and SVM) and using them as an estimator into voting classification(Ensamble method)
    
    Total 4 classifiers are created and I used HARD voting classification here.

In [ ]:
#fit the data into different models

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time

# random-forests-classifier
rf_clf = RandomForestClassifier(max_depth=10, random_state=42)

#extra-tree-classifier
et_classifier = ExtraTreesClassifier(n_estimators=10, random_state=42)

#svm-classifiers
svm_clf = SVC(gamma=100.0, C=1.0, random_state=42)

#voting-classifiers
voting_clf = VotingClassifier(
    estimators=[('etc', et_classifier), ('rfc', rf_clf), ('svc', svm_clf)],
    voting='hard'
)

## Step 5 - Test the models with score/accuracy
     All above created models are fit with training data, and try to findout the score value on validation and test dataset.
     1. I Used dictionary to store all the classifiers and looping them through to get one-by-one value.
     2. All the models produced score values 0.8517, 0.9046, 0.1152 and 0.9176 respectively.
     3. We can clearly say that voting classifer have better results than all other classifiers.

In [ ]:
clf_list = {"random_forest_classifier": rf_clf, "extra-tree-classifier":et_classifier, 
            "svm_classifier": svm_clf, "voting_classifier":voting_clf}

for name, cassification in clf_list.items() :
    print("--------", name, "-----------")
    cassification.fit(train_components, y_train)
    y_pred = cassification.predict(test_components)
    print("Validation Score", cassification.score(validate_components, y_validation))
    print("Prediction ", cassification.score(test_components, y_pred)*100 ,"%")


## Additional step
    I added another way to find out the score value that can be applicable on whole features instead of filtered components. The accuracy that I got here are almost similar(slightly higher for all).

In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(et_classifier, validate_components, y_validation, scoring='accuracy', cv=5))
print(cross_val_score(rf_clf, validate_components, y_validation, scoring='accuracy', cv=5))
print(cross_val_score(svm_clf, validate_components, y_validation, scoring='accuracy', cv=5))
print(cross_val_score(voting_clf, validate_components, y_validation, scoring='accuracy', cv=5))

print(cross_val_score(et_classifier, X_test, y_test, scoring='accuracy', cv=5))
print(cross_val_score(rf_clf, X_test, y_test, scoring='accuracy', cv=5))
print(cross_val_score(svm_clf, X_test, y_test, scoring='accuracy', cv=5))
print(cross_val_score(voting_clf, X_test, y_test, scoring='accuracy', cv=5))

## Question 2 Principle Component Analysis
The following code generates 40 3-dimensional samples randomly drawn from a (multivariate) gaussian
distribution.  This simply means that we have 3 variables, or dimensions.  Each dimension for the 40
samples is drawn with some sample mean, and some standard deviation.  We generate 2 different classes
where one half (i.e. 20) samples of our data are from class 1 and the other half are from class 2.
We use the following sample means and covariance matrices to generate the data:

$$
\mu_1 =
\begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix}
\mu_2 =
\begin{bmatrix}
1 \\
1 \\
1 \\
\end{bmatrix}
\textrm{(sample means)}
$$

$$
\Sigma_1 =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
\Sigma_2 =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
\textrm{(covariance matrices)}
$$

This basically means the mean value for each of the 3 dimensions for the class 1 is 0, and for class 2
it is 1.  The covariance matrices are similar to the ones we discussed a bit when looking at PCA, for
now you can simply think of them as specifying the amount of standard deviation we will have for each
dimension.  Here is the code to generate two $20 \times 3$ classes of data:

In [ ]:
np.random.seed(1) # random seed for consistency

mu_vec1 = np.array([0,0,0])
cov_mat1 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class1_sample = np.random.multivariate_normal(mu_vec1, cov_mat1, 20)
assert class1_sample.shape == (20,3), "The matrix has not the dimensions 20x3"

mu_vec2 = np.array([1,1,1])
cov_mat2 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class2_sample = np.random.multivariate_normal(mu_vec2, cov_mat2, 20)
assert class1_sample.shape == (20, 3), "The matrix has not the dimensions 20x3"

Just to get a rough idea of how the samples of our two classes are distributed, let us plot them in
a 3D scatter plot.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10
ax.plot(class1_sample[:,0], class1_sample[:,1], class1_sample[:,2],
        'o', markersize=8, color='blue', alpha=0.5, label='class1')
ax.plot(class2_sample[:,0], class2_sample[:,1], class2_sample[:,2],
        '^', markersize=8, alpha=0.5, color='red', label='class2')

plt.title('Samples for class 1 and class 2')
ax.legend(loc='upper right')


Here we will concatenate both of our 20 class samples into a single numpy array

In [ ]:
all_samples = np.concatenate((class1_sample, class2_sample), axis=0)
assert all_samples.shape == (40,3), "The matrix has not the dimensions 3x40"

Using the methods we did in our lectures and assignments, perform a principal component analysis
(PCA) on the combined set of 40 samples of data.  Reduce the dimensions too 2 dimensions and plot
the resulting PCA dimensionality reduction on a 2D figure.

In [ ]:
# step 1, normalize the features...

mean1 = np.mean(all_samples)

mu = np.mean(all_samples, axis=0)
sd =  np.std(all_samples, axis=0)

X_norm = all_samples - mu
X_norm_scaled = (all_samples - mu) / sd

# extract m, number of samples
# extract n, number of features/dimensions
m,n = X_norm_scaled.shape
print(m,n)

In [ ]:
# step 2, do the PCA
def pca(X):
    Sigma = np.empty( (n, n) )
    U = S = V = np.zeros( (n, n) )
    
    Sigma = np.dot(X.T , X) / m
    U , S, V = np.linalg.svd(Sigma)
    
    # return the U matrix of the principal components, and the S vector of the variance measures
    return U, S

U, S = pca(X_norm_scaled)
print(U)
print(S)



In [ ]:
# step 3, reduce to 2 dimensions and project back onto the 2 dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=2, whiten=True)
pca.fit(X_norm_scaled)
print(pca.components_)
print(pca.explained_variance_)

X_pca = pca.transform(X_norm_scaled)
print("original shape:   ", X_norm_scaled.shape)
print("transformed shape:", X_pca.shape)

In [ ]:
# finally, plot the recovered data
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10
ax.plot(all_samples[:,0],all_samples[:,1],
        'o', markersize=8, color='blue', alpha=0.5, label='PCA-Dim-Reduction')

plt.title('3D-To-2D-Dimension-reduction')
ax.legend(loc='lower left')

